# Assignments and Real Problems
Practice problems based on real-world patterns. Try to solve without looking up answers.

## Assignment 1: Data normalization (tabular)
Given raw features, standardize each column to zero mean and unit variance.

In [6]:
from numpy import mean
import torch

# Example data: rows are samples, columns are features
X = torch.tensor([
    [1.0, 200.0, 0.5],
    [2.0, 180.0, 0.3],
    [3.0, 210.0, 0.9],
    [4.0, 190.0, 0.1],
])

# TODO: standardize columns
# mean = ?
# std = ?
# X_std = ?

mean = X.mean(dim=0) # what is this? dim = 0 means we are computing the mean for each feature (column)
std = X.std(dim=0) # what is this? dim = 0 means we are computing the std for each feature (column)
X_std = (X - mean) / std

# Print standardized data
print(X_std)

tensor([[-1.1619,  0.3873,  0.1464],
        [-0.3873, -1.1619, -0.4392],
        [ 0.3873,  1.1619,  1.3175],
        [ 1.1619, -0.3873, -1.0247]])


## Assignment 2: Linear regression for house prices
Predict house price from size and number of rooms using a linear model.

In [11]:
from math import e
from requests import get
import torch
import torch.nn as nn

# Features: [size (sq ft), rooms]
X = torch.tensor([
    [800.0, 2.0],
    [1000.0, 3.0],
    [1200.0, 3.0],
    [1500.0, 4.0],
    [1800.0, 4.0],
])

# Prices in thousands
Y = torch.tensor([[150.0], [200.0], [230.0], [280.0], [320.0]])

# TODO: build a linear model, train for 200 epochs, print final loss
# model = ?
# criterion = ?
# optimizer = ?

class SimpleLinearRegression(nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

model = SimpleLinearRegression(input_dim=2, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Save the model
torch.save(model.state_dict(), "simple_linear_regression.pth")
# what is pth?
# .pth is a file extension used by PyTorch to indicate a file that contains a serialized model or tensor.
# use it to save and load models
# load to load the model
# model = SimpleLinearRegression(input_dim=2, output_dim=1) # why this ? We need to create a new instance of the model to load the saved parameters into.
model.load_state_dict(torch.load("simple_linear_regression.pth")) # what is this? This loads the model's parameters from the specified file.

def get_parameters(model):
    return {name: param.data for name, param in model.named_parameters()}

get_parameters(model)

Epoch [20/200], Loss: 6827.9741
Epoch [40/200], Loss: 927.8101
Epoch [60/200], Loss: 151.1384
Epoch [80/200], Loss: 120.3104
Epoch [100/200], Loss: 92.8200
Epoch [120/200], Loss: 92.0170
Epoch [140/200], Loss: 91.9770
Epoch [160/200], Loss: 91.9053
Epoch [180/200], Loss: 91.8610
Epoch [200/200], Loss: 91.8205


{'linear.weight': tensor([[0.1851, 0.3658]]), 'linear.bias': tensor([-0.3361])}